In [1]:
import random

with open('/kaggle/input/assignmentcl/test.en', 'r') as f1:
    en_lines = f1.readlines()

with open('/kaggle/input/assignmentcl/test.hi', 'r') as f2:
    hi_lines = f2.readlines()
    
with open('/kaggle/input/assignmentcl/test.ta', 'r') as f3:
    ta_lines = f3.readlines()

random_indices = random.sample(range(len(en_lines)), 1000)

selected_en = [en_lines[i] for i in random_indices]
selected_hi = [hi_lines[i] for i in random_indices]
selected_ta = [ta_lines[i] for i in random_indices]

output_file1 = '/kaggle/working/enselected.txt'
with open(output_file1, 'w') as f1:
    for line in selected_en:
        f1.write(line)
        
output_file2 = '/kaggle/working/hiselected.txt'
with open(output_file2, 'w') as f2:
    for line in selected_hi:
        f2.write(line)
        
output_file3 = '/kaggle/working/taselected.txt'
with open(output_file3, 'w') as f3:
    for line in selected_ta:
        f3.write(line)

## ENGLISH TO HINDI

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M", src_lang="eng_Latn"
)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [3]:
model = model.to("cuda")

In [4]:
answers=[]
for line in selected_en:
    inputs = tokenizer(line, return_tensors="pt").to("cuda")

    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["hin_Deva"], max_length=30
    )
    x = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    answers.append(x)

the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38


In [5]:
output_file = 'en_to_hi_output.txt'
with open(output_file, 'w') as f:
    for line in answers:
        f.write(line+"\n")

## HINDI TO ENGLISH

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M", src_lang="hin_Deva"
)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
model = model.to("cuda")

answers=[]
for line in selected_hi:
    inputs = tokenizer(line, return_tensors="pt").to("cuda")

    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["eng_Latn"], max_length=30
    )
    x = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    answers.append(x)

In [7]:
output_file = 'hi_to_en_output.txt'
with open(output_file, 'w') as f:
    for line in answers:
        f.write(line+"\n")

## TAMIL TO HINDI

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M", src_lang="tam_Taml"
)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
model = model.to("cuda")
answers=[]
for line in selected_ta:
    inputs = tokenizer(line, return_tensors="pt").to("cuda")

    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["hin_Deva"], max_length=30
    )
    x = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    answers.append(x)

In [9]:
output_file = 'ta_to_hi_output.txt'
with open(output_file, 'w') as f:
    for line in answers:
        f.write(line+"\n")

## HINDI TO TAMIL

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "facebook/nllb-200-distilled-600M", src_lang="hin_Deva"
)
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
model = model.to("cuda")
answers=[]
for line in selected_hi:
    inputs = tokenizer(line, return_tensors="pt").to("cuda")

    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["tam_Taml"], max_length=30
    )
    x = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    answers.append(x)

In [11]:
output_file = 'hi_to_ta_output.txt'
with open(output_file, 'w') as f:
    for line in answers:
        f.write(line+"\n")

## BLEU CALCULATION

In [1]:
with open('/kaggle/input/outputfiles/enselected.txt', 'r') as f1:
    en = f1.readlines()

with open('/kaggle/input/outputfiles/hiselected.txt', 'r') as f2:
    hi = f2.readlines()

with open('/kaggle/input/outputfiles/taselected.txt', 'r') as f2:
    ta = f2.readlines()
    
with open('/kaggle/input/outputfiles/en_to_hi_output.txt', 'r') as f1:
    en_to_hi = f1.readlines()
    
with open('/kaggle/input/outputfiles/hi_to_en_output.txt', 'r') as f1:
    hi_to_en = f1.readlines()

with open('/kaggle/input/outputfiles/ta_to_hi_output.txt', 'r') as f1:
    ta_to_hi = f1.readlines()
    
with open('/kaggle/input/outputfiles/hi_to_ta_output.txt', 'r') as f1:
    hi_to_ta = f1.readlines()

In [2]:
from nltk.translate.bleu_score import corpus_bleu

In [3]:
bleu_score = corpus_bleu(en[:50], hi_to_en[:50])
print("BLEU SCORE FOR HINDI TO ENGLISH: ",bleu_score)

BLEU SCORE FOR HINDI TO ENGLISH:  0.706722337625123


/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [4]:
bleu_score = corpus_bleu(hi[:50], en_to_hi[:50])
print("BLEU SCORE FOR ENGLISH TO HINDI: ",bleu_score)

BLEU SCORE FOR ENGLISH TO HINDI:  0.7547235743984803


In [5]:
bleu_score = corpus_bleu(hi[:50], ta_to_hi[:50])
print("BLEU SCORE FOR TAMIL TO HINDI: ",bleu_score)

BLEU SCORE FOR TAMIL TO HINDI:  0.7513221168058316


In [6]:
bleu_score = corpus_bleu(ta[:50], hi_to_ta[:50])
print("BLEU SCORE FOR HINDI TO TAMIL: ",bleu_score)

BLEU SCORE FOR HINDI TO TAMIL:  0.7246769061056083


## ROUGE SCORE

In [7]:
!pip install rouge

In [8]:
from rouge import Rouge
rouge = Rouge()

In [9]:
scores = rouge.get_scores(hi_to_en[:50], en[:50], avg=True)
print("ROUGE SCORE FOR HINDI TO ENGLISH: ")
for key,value in scores.items():
    print(key, ":" ,value)

ROUGE SCORE FOR HINDI TO ENGLISH: 
rouge-1 : {'r': 0.6231743845998314, 'p': 0.6505846348055258, 'f': 0.6320877570711185}
rouge-2 : {'r': 0.3914939360861491, 'p': 0.41697247779590624, 'f': 0.3999141372832685}
rouge-l : {'r': 0.59415269188886, 'p': 0.6205815408660676, 'f': 0.6027146517216301}


In [10]:
scores = rouge.get_scores(en_to_hi[:50], hi[:50], avg=True)
print("ROUGE SCORE FOR ENGLISH TO HINDI: ")
for key,value in scores.items():
    print(key, ":" ,value)

ROUGE SCORE FOR ENGLISH TO HINDI: 
rouge-1 : {'r': 0.5392008199677981, 'p': 0.5947897027293315, 'f': 0.5600182291575057}
rouge-2 : {'r': 0.31336652698530126, 'p': 0.3457104125253608, 'f': 0.3248774584472058}
rouge-l : {'r': 0.5077184900641124, 'p': 0.5617706259169108, 'f': 0.5280956735728352}


In [11]:
scores = rouge.get_scores(ta_to_hi[:50], hi[:50], avg=True)
print("ROUGE SCORE FOR TAMIL TO HINDI: ")
for key,value in scores.items():
    print(key, ":" ,value)

ROUGE SCORE FOR TAMIL TO HINDI: 
rouge-1 : {'r': 0.42326570792801804, 'p': 0.4761380325900449, 'f': 0.44299992792432796}
rouge-2 : {'r': 0.21631379362848605, 'p': 0.2408738642603563, 'f': 0.2249303555799263}
rouge-l : {'r': 0.3899729295015069, 'p': 0.4352291041431907, 'f': 0.40666850160880086}


In [12]:
scores = rouge.get_scores(hi_to_ta[:50], ta[:50], avg=True)
print("ROUGE SCORE FOR HINDI TO TAMIL: ")
for key,value in scores.items():
    print(key, ":" ,value)

ROUGE SCORE FOR HINDI TO TAMIL: 
rouge-1 : {'r': 0.32642052986276854, 'p': 0.37399252055134413, 'f': 0.34279389677833094}
rouge-2 : {'r': 0.1008771827587617, 'p': 0.11106413031413034, 'f': 0.10374456566245559}
rouge-l : {'r': 0.31359133938905487, 'p': 0.35819914725797086, 'f': 0.3289576833185881}
